In [4]:
import math
import pandas as pd
import numpy as np
# Haversine distance calculations
def haversine_distance(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371  # Earth's radius in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance1 = R * c
    distance = distance1 * 0.62137  # Convert KM to M
    return distance

# Read data from Excel files into pandas DataFrames
shipment_destinations = pd.read_excel('cwyh.xlsx')
distribution_centers = pd.read_excel('distribution_centers.xlsx')

nearest_dc = []  # Create variable called 'nearest_dc' to hold both lists

for index1, destination in shipment_destinations.iterrows():
    min_distance = float('inf')
    nearest_center = None

    for index2, center in distribution_centers.iterrows():
        distance = haversine_distance(destination[['Latitude', 'Longitude']], center[['Latitude', 'Longitude']])

        if distance < min_distance:
            min_distance = distance
            nearest_center = center['id']

    nearest_dc.append({'destination_id': destination['Name'], 'nearest_dc_id': nearest_center, 'distance': min_distance})

print(nearest_dc)  # Show results of the function

[{'destination_id': 30303, 'nearest_dc_id': 'Target LOC SOCS', 'distance': 0.36839281077924196}, {'destination_id': 91752, 'nearest_dc_id': 'Target LOC T9479', 'distance': 5.5659333715554755}, {'destination_id': 7106, 'nearest_dc_id': 'Target LOC T3687', 'distance': 21.334128259562178}, {'destination_id': 90746, 'nearest_dc_id': 'Target LOC YWDC', 'distance': 16.696706416917767}, {'destination_id': 15104, 'nearest_dc_id': 'Target LOC T0589', 'distance': 155.6567440454089}, {'destination_id': 83402, 'nearest_dc_id': 'Target LOC T0558', 'distance': 332.38870439984544}, {'destination_id': 24112, 'nearest_dc_id': 'Target LOC T0560', 'distance': 29.106018197795482}, {'destination_id': 85621, 'nearest_dc_id': 'Target LOC T9478', 'distance': 55.50964336602096}, {'destination_id': 92374, 'nearest_dc_id': 'Target LOC T9479', 'distance': 9.322745048446963}, {'destination_id': 37066, 'nearest_dc_id': 'Target LOC T0580', 'distance': 113.66291183312767}, {'destination_id': 95376, 'nearest_dc_id': '

In [5]:
import plotly.express as px
df= pd.DataFrame(nearest_dc) #create dataframe holding our function to use in visualization portion of model
df1= pd.DataFrame(shipment_destinations, distribution_centers) #create dataframe holding our function to use in analysis visualization portion of model
#df.head()
df1.head()

,Name,Latitude,Longitude,KMA,Volume
"(CCXP, 31407.0, nan, 32.0187, -81.0967, nan, Target LOC CCXP, Shipper , 5.0, 5, NY_BRN, Brooklyn Mkt, Brooklyn, NY, Northeast, 11747, nan, nan, 5)",NaN,NaN,NaN,NaN,NaN
"(CCXV, 23434.0, nan, 36.8133, -76.3079, nan, Target LOC CCXV, Shipper , nan, 10, MA_SPR, Springfield Mkt, Springfield, MA, Northeast, 1373, nan, nan, 10)",NaN,NaN,NaN,NaN,NaN
"(YLAW, 98390.0, nan, 47.2458, -123.149, nan, Target LOC YLAW, Shipper , nan, 11, MA_SPR, Springfield Mkt, Springfield, MA, Northeast, 1373, nan, nan, 11)",NaN,NaN,NaN,NaN,NaN
"(YWDC, 90810.0, nan, 33.8183, -118.3517, nan, Target LOC YWDC, Shipper , nan, 12, MA_SPR, Springfield Mkt, Springfield, MA, Northeast, 1373, nan, nan, 12)",NaN,NaN,NaN,NaN,NaN
"(YLTA, 29407.0, nan, 32.8014, -80.0258, nan, Target LOC YLTA, Shipper , nan, 13, MA_SPR, Springfield Mkt, Springfield, MA, Northeast, 1373, nan, nan, 13)",NaN,NaN,NaN,NaN,NaN


In [10]:
#create analytical variables for general information and creation of weighted score downstream
dist_col = df['distance']
dist_col = dist_col[~np.isnan(dist_col)] # remove missing values
dist_col = dist_col[dist_col != np.inf] # remove infinite values
max_dist = dist_col.max()
mean_dist = dist_col.mean()
min_dist = dist_col.min()
max_dist = dist_col.max()
mean_dist =  dist_col.mean()
min_dist = dist_col.min()
distance_weight = .9 #weight assigned to distance 
volume_weight = 0.1 #weight assigned to volume

#show results
print('Average distance:', mean_dist)
print('Minimum distance:', min_dist)
print('Maximum distance:', max_dist)

Average distance: 90.54804220804165
Minimum distance: 0.36839281077924196
Maximum distance: 433.1732973793138


In [12]:
mean_vol = 1119
max_volume = 2383
score2 = 1 - ((dist_col / max_dist) * distance_weight + (mean_vol / max_volume) * volume_weight)
score2 = score2.clip(lower=.0)
print(score2)
mean_score2 = np.mean(score2)

print("Average FIT Score for Top lanes:", mean_score2) #.765

0     0.952277
1     0.941478
2     0.908717
3     0.918352
4     0.629636
5     0.262442
6     0.892569
7     0.837711
8     0.933673
9     0.716886
10    0.835453
11    0.926330
12    0.579067
13    0.927635
14    0.666975
15    0.849651
16    0.272844
17    0.053042
18    0.545628
19    0.895426
20    0.867537
21    0.897464
22    0.842420
23    0.841759
24    0.944443
25    0.948290
Name: distance, dtype: float64
Average FIT Score for Top lanes: 0.7649115858295364


In [13]:
import plotly.graph_objects as go
import pandas as pd
data = shipment_destinations

In [19]:
fig = go.Figure(data=go.Choropleth(
    locations=data['Name'],
    locationmode='USA-states',
    z=data['Volume']
))
fig.show()